# 릿지(alpha=0.1)로 선택한 변수로 돌린 모델(ver4.0)

'생년', '월 평균 총가구소득', '키(센티)', '몸무게(kg)', '가구 유형', '가장 불편한부위',
        '장애주된 원인', '질병명_38', '주된 진단명_40',
       '건강보험가입여부 및 형태', '사고발생내용2',  '희귀난치성질환 등록 여부',
       '암 종류1', '암 종류2',  '지난 2주간 아팠던 날수', '언어치료-이용여부',
       '놀이치료-이용여부', '심리행동치료-이용여부',
        'ADL - 구강청결', 'ADL –음식물 넘기기', 'ADL - 옮겨앉기',
       'ADL– 배변', 'ADL– 빨래하기', 'ADL –약 챙겨먹기', 
       '현재 일상생활 도와주는 이유무', '주로 도와주는 사람', '팔받침대-사용',
       '의사소통 보조기기-필요', '특수마우스-필요',
       '이동식 전동리프트-필요'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_excel("지체장애_고혈압_결측치 대체_데이터.xlsx")
df.head()

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 주된 수입원,가구 월평균 지출액,장애발생시 연령_36,월 혈압약 일수,...,인공호흡기-소지_673,인공호흡기-사용_674,사용빈도,1일 이용시간,이용경험-상담과 평가,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,인터넷 사용 여부,01)만성질환명(고혈압)
0,1,1970,2008,3,1,280,1,250,38,0,...,2,0,0,0,2,2,1,1,1,0
1,2,1949,2010,3,1,180,3,150,20,30,...,2,0,0,0,2,1,2,2,2,1
2,4,1958,2011,2,1,350,1,200,51,0,...,2,0,0,0,2,2,1,2,2,0
3,5,1955,2009,3,2,154,1,154,53,30,...,2,0,1,3,2,2,1,2,2,1
4,6,1936,2010,2,1,275,3,270,62,0,...,2,0,2,9,1,1,2,2,2,0


In [4]:
df.columns

Index(['id', '생년', '장애등록 연도', '본인을 포함한 총 가구원수', '본인을 포함한 총 장애인수', '월 평균 총가구소득',
       '가구 주된 수입원', '가구 월평균 지출액', '장애발생시 연령_36', '월 혈압약 일수',
       ...
       '인공호흡기-소지_673', '인공호흡기-사용_674', '사용빈도', '1일 이용시간', '이용경험-상담과 평가',
       '휴대폰 사용 여부', '스마트폰 사용 여부', '컴퓨터 사용 여부', '인터넷 사용 여부', '01)만성질환명(고혈압)'],
      dtype='object', length=267)

In [10]:
df1=df[['생년', '월 평균 총가구소득', '키(센티)', '몸무게(kg)', '가구 유형', '가장 불편한부위',
        '장애주된 원인', '질병명_38', '주된 진단명_40',
       '건강보험가입여부 및 형태', '사고발생내용2',  '희귀난치성질환 등록 여부',
       '암 종류1', '암 종류2',  '지난 2주간 아팠던 날수', '언어치료-이용여부', '놀이치료-이용여부', '심리행동치료-이용여부',
        'ADL - 구강청결', 'ADL –음식물 넘기기', 'ADL - 옮겨앉기',
       'ADL– 배변', 'ADL– 빨래하기', 'ADL –약 챙겨먹기', 
       '현재 일상생활 도와주는 이유무', '주로 도와주는 사람', '팔받침대-사용',
       '의사소통 보조기기-필요', '특수마우스-필요',
       '이동식 전동리프트-필요','01)만성질환명(고혈압)']]
df1 #2686건

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요,01)만성질환명(고혈압)
0,1970,280,178.0,77.0,21,36,64,0,12,1,...,1,1,1,0,0,0,2,2,2,0
1,1949,180,163.0,55.0,21,14,69,0,8,1,...,1,2,1,1,1,0,2,2,2,1
2,1958,350,158.0,63.0,11,38,48,175,11,1,...,1,1,1,0,0,0,2,2,2,0
3,1955,154,163.0,62.0,29,38,48,175,11,3,...,1,1,1,0,0,0,2,2,2,1
4,1936,275,154.0,57.0,24,36,48,171,6,1,...,3,4,2,1,13,0,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,1935,100,157.0,65.0,11,36,68,0,12,2,...,1,2,1,1,1,0,2,2,2,0
2682,1952,100,165.0,62.0,11,38,69,0,12,2,...,1,1,1,0,0,0,2,2,2,0
2683,1960,223,173.0,88.0,11,38,48,175,11,2,...,1,1,1,0,0,0,2,2,2,1
2684,1960,205,153.0,51.0,11,31,48,175,11,1,...,1,1,1,0,0,0,2,2,2,0


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [11]:
df1[df1['01)만성질환명(고혈압)']==1]

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요,01)만성질환명(고혈압)
1,1949,180,163.0,55.0,21,14,69,0,8,1,...,1,2,1,1,1,0,2,2,2,1
3,1955,154,163.0,62.0,29,38,48,175,11,3,...,1,1,1,0,0,0,2,2,2,1
5,1938,160,155.0,60.0,11,38,63,0,12,2,...,1,2,1,1,14,0,2,2,2,1
6,1943,600,145.0,45.0,32,14,69,0,8,1,...,1,2,1,1,3,0,2,2,2,1
7,1945,270,155.0,70.0,11,36,48,171,6,2,...,1,2,1,2,0,0,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,1951,225,167.0,59.0,11,36,48,171,6,1,...,1,1,1,0,0,0,2,2,2,1
2679,1938,93,160.0,39.0,11,38,69,0,12,3,...,1,2,1,1,12,0,2,2,2,1
2680,1951,93,145.0,55.0,11,45,61,0,6,3,...,1,2,1,1,12,0,2,2,2,1
2683,1960,223,173.0,88.0,11,38,48,175,11,2,...,1,1,1,0,0,0,2,2,2,1


In [60]:
out=df1.loc[[50]] # 고혈압 3, 32유 , 46,50무  
out

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요,01)만성질환명(고혈압)
50,1970,1136,172.0,58.0,21,38,48,175,11,2,...,1,1,1,0,0,0,2,2,2,0


In [61]:
df2=df1.drop(out.index).reset_index(drop=True)
df2.head()

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요,01)만성질환명(고혈압)
0,1970,280,178.0,77.0,21,36,64,0,12,1,...,1,1,1,0,0,0,2,2,2,0
1,1949,180,163.0,55.0,21,14,69,0,8,1,...,1,2,1,1,1,0,2,2,2,1
2,1958,350,158.0,63.0,11,38,48,175,11,1,...,1,1,1,0,0,0,2,2,2,0
3,1955,154,163.0,62.0,29,38,48,175,11,3,...,1,1,1,0,0,0,2,2,2,1
4,1936,275,154.0,57.0,24,36,48,171,6,1,...,3,4,2,1,13,0,2,2,2,0


In [62]:
y_data=df2[['01)만성질환명(고혈압)']]

In [63]:
x_data=df2.drop(['01)만성질환명(고혈압)'],axis=1)
x_data

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL - 옮겨앉기,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요
0,1970,280,178.0,77.0,21,36,64,0,12,1,...,1,1,1,1,0,0,0,2,2,2
1,1949,180,163.0,55.0,21,14,69,0,8,1,...,1,1,2,1,1,1,0,2,2,2
2,1958,350,158.0,63.0,11,38,48,175,11,1,...,1,1,1,1,0,0,0,2,2,2
3,1955,154,163.0,62.0,29,38,48,175,11,3,...,1,1,1,1,0,0,0,2,2,2
4,1936,275,154.0,57.0,24,36,48,171,6,1,...,3,3,4,2,1,13,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1935,100,157.0,65.0,11,36,68,0,12,2,...,1,1,2,1,1,1,0,2,2,2
2681,1952,100,165.0,62.0,11,38,69,0,12,2,...,1,1,1,1,0,0,0,2,2,2
2682,1960,223,173.0,88.0,11,38,48,175,11,2,...,1,1,1,1,0,0,0,2,2,2
2683,1960,205,153.0,51.0,11,31,48,175,11,1,...,1,1,1,1,0,0,0,2,2,2


In [64]:
y_data.value_counts()

01)만성질환명(고혈압)
1                1528
0                1157
dtype: int64

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [66]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['가구 유형', '가장 불편한부위',
        '장애주된 원인', '질병명_38', '주된 진단명_40',
       '건강보험가입여부 및 형태', '사고발생내용2',  '희귀난치성질환 등록 여부',
       '암 종류1', '암 종류2',  '지난 2주간 아팠던 날수', '언어치료-이용여부',
       '놀이치료-이용여부', '심리행동치료-이용여부',
        'ADL - 구강청결', 'ADL –음식물 넘기기', 'ADL - 옮겨앉기',
       'ADL– 배변', 'ADL– 빨래하기', 'ADL –약 챙겨먹기', 
       '현재 일상생활 도와주는 이유무', '주로 도와주는 사람', '팔받침대-사용',
       '의사소통 보조기기-필요', '특수마우스-필요',
       '이동식 전동리프트-필요']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_train)
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [67]:
if not os.path.exists('models/disabled_chronic_disease/'):
    os.makedirs('models/disabled_chronic_disease/')

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(transformer, f)

In [68]:
#모델 생성

model = SVC(kernel='rbf', C=5.0, gamma=0.01, probability=True)#,class_weight=class_weight)

In [69]:
#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('training data fit :', model.score(x_train, y_train)) 

print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\Users\LJM\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


training data fit : 0.6726982437466738
test data fit : 0.6563275434243176


In [70]:
x_test=out.drop(['01)만성질환명(고혈압)'],axis=1)
x_test

,생년,월 평균 총가구소득,키(센티),몸무게(kg),가구 유형,가장 불편한부위,장애주된 원인,질병명_38,주된 진단명_40,건강보험가입여부 및 형태,...,ADL - 옮겨앉기,ADL– 배변,ADL– 빨래하기,ADL –약 챙겨먹기,현재 일상생활 도와주는 이유무,주로 도와주는 사람,팔받침대-사용,의사소통 보조기기-필요,특수마우스-필요,이동식 전동리프트-필요
50,1970,1136,172.0,58.0,21,38,48,175,11,2,...,1,1,1,1,0,0,0,2,2,2


In [71]:
labels = ['무', '유']

In [72]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

유 0.5888767447743728


---